In [22]:
import pandas as pd
import numpy as np

from urllib.parse import quote    
from urllib.request import urlopen

import sys
import requests
from bs4 import BeautifulSoup 
import pandas as pd
from sqlalchemy import create_engine

#import pymysql.cursors
#pymysql.install_as_MySQLdb()
#import MySQLdb
from pandas.io.json import json_normalize
import time
import datetime
from datetime import timedelta
import re
import folium
from folium.plugins import MarkerCluster, MiniMap

from selenium import webdriver
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.keys import Keys

In [18]:
auto1 = 'ember1603'
auto2 = '_1599991075864'

In [23]:
base_url = 'https://www.arcgis.com/apps/opsdashboard/index.html#/bda7594740fd40299423467b48e9ecf6'
browser = webdriver.Chrome('/Users/sinmin-yong/GitHub/공모전/도서빅데이터/chromedriver')
time.sleep(0.5)
browser.get(base_url)

In [28]:
span_ls = browser.find_elements_by_xpath('//nav[@class = "feature-list"]')[0].find_elements_by_xpath('span')

In [38]:
country_nm = []

for nm in span_ls:
    country_nm.append(' '.join(nm.text.split(' ')[1:]))

In [46]:
country_df = pd.DataFrame()
country_df['country'] = country_nm
country_df['id'] = list(range(len(span_ls)))

In [51]:
covid_df = pd.DataFrame()
for span,ctm in zip(span_ls,country_nm):
    
    span.click()
    
    time.sleep(1.5)
    
    html = browser.page_source
    time.sleep(0.5)
    
    soup = BeautifulSoup(html,'html.parser')
    patient_ls = soup.find('g',{'class':re.compile('amcharts-graph-column amcharts-graph-graphAuto0')})
    patient_ls2 = patient_ls.find_all('g')[1:]
    
    sick_ls = []
    month_ls = []
    day_ls = []
    
    for idx,patient in enumerate(patient_ls2):

        sick_ls.append(patient['aria-label'].split(' ')[-1])
        month_ls.append(patient['aria-label'].split(' ')[1])
        day_ls.append(patient['aria-label'].split(' ')[2])

    day_df = pd.Series(day_ls).apply(lambda x : re.sub('[^0-9]','',x))
    month_df = pd.Series(month_ls).apply(lambda x : re.sub('[^0-9]','',x))
    month_df = month_df.apply(lambda x : '0'+x if len(x)==1 else x)

    ym_df = pd.Series(['2020']*len(month_df)) + pd.Series(['-']*len(month_df))+ month_df  + pd.Series(['-']*len(month_df)) + day_df

    covid_df0 = pd.DataFrame()

    covid_df0['date'] = ym_df
    covid_df0['patient'] = sick_ls
    covid_df0['country'] = ctm
    
    covid_df = covid_df.append(covid_df0)

In [54]:
covid_df2 = pd.merge(covid_df,country_df,on = 'country', how = 'left').drop(columns = 'country')

In [57]:
covid_df2 = covid_df2.rename(columns = {'id' : 'country_id'})

In [59]:
covid_df2.to_csv('data/worldwide_covid19.csv',index=False)
country_df.to_csv('data/worldwide_country.csv',index=False)

In [21]:
covid_df.to_csv('data/covid19_data.csv',index = False)